# Install dependencies

In [1]:
!pip install -qU langchain langchain-core langchain-community langchain-openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install -qU tiktoken pymupdf


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install -qU ragas


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install -qU faiss_cpu


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Set enviroment values

In [5]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

Please provide your OpenAI Key:  ········


In [6]:
os.chdir('mlops-course/AI-Engineering/MidtermChallenge')

# Load document

In [7]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(
    "Nvidia_10k_Filings.pdf",
)

documents = loader.load()

In [13]:
documents[0].metadata

{'source': 'Nvidia_10k_Filings.pdf',
 'file_path': 'Nvidia_10k_Filings.pdf',
 'page': 0,
 'total_pages': 96,
 'format': 'PDF 1.4',
 'title': '0001045810-24-000029',
 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group',
 'subject': 'Form 10-K filed on 2024-02-21 for the period ending 2024-01-28',
 'keywords': '0001045810-24-000029; ; 10-K',
 'creator': 'EDGAR Filing HTML Converter',
 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0',
 'creationDate': "D:20240221173732-05'00'",
 'modDate': "D:20240221173744-05'00'",
 'trapped': '',
 'encryption': 'Standard V2 R3 128-bit RC4'}

# Document chunks

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

In [9]:
len(documents)

624

# Load embedding model

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

# Create FAISS VectorStore and Retriever

In [11]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

In [12]:
retriever = vector_store.as_retriever()

# Test the Retriever

In [13]:
retrieved_documents = retriever.invoke("What is Ndivia?")

In [20]:
for doc in retrieved_documents:
  print(doc)

page_content='expanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for accelerated\ncomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are "Compute & Networking" and "Graphics." Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,\nItem 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nRecent Developments, Future Objectives and Challenges' metadata={'source': 'Nvidia_10k_Filings.pdf', 'file_path': 'Nvidia_10k_Filings.pdf', 'page': 33, 'total_pages': 96, 'format': 'PDF 1.4', 'title': '0001045810-24-000029', 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'subject': 'Form 10-K filed on 2024-02-21 for the per

# Create the RAG Chain

In [14]:
!pip install langchainhub


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [16]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


In [17]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [19]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

# Testing the chain

In [20]:
question = "What is Nvidia?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

Nvidia is a company that has leveraged its GPU architecture to create platforms for accelerated computing, AI solutions, scientific computing, data science, AV, robotics, metaverse, and 3D internet applications.


### Pretty cool but the next one is dissapointing

In [21]:
question = "What is this document about?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(page_content='32.1#*\nCertification of Chief Executive Officer as required by Rule 13a-14(b) of the Securities Exchange Act of 1934\n32.2#*\nCertification of Chief Financial Officer as required by Rule 13a-14(b) of the Securities Exchange Act of 1934\n97.1+*\nCompensation Recovery Policy, as amended and restated November 30, 2023\n101.INS*\nXBRL Instance Document\n101.SCH*\nXBRL Taxonomy Extension Schema Document\n101.CAL*\nXBRL Taxonomy Extension Calculation Linkbase Document\n101.DEF*\nXBRL Taxonomy Extension Definition Linkbase Document\n101.LAB*\nXBRL Taxonomy Extension Labels Linkbase Document\n101.PRE*\nXBRL Taxonomy Extension Presentation Linkbase Document\n104', metadata={'source': 'Nvidia_10k_Filings.pdf', 'file_path': 'Nvidia_10k_Filings.pdf', 'page': 82, 'total_pages': 96, 'format': 'PDF 1.4', 'title': '0001045810-24-000029', 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'subject': 'Form 10-K filed on 2024-02-21 for the period endi

# Synthetic Dataset Generation using Ragas

In [23]:
eval_documents = documents

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

eval_documents = text_splitter.split_documents(eval_documents)

In [26]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.with_openai()

testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.25, reasoning: 0.25, multi_context: 0.5},raise_exceptions=False)
     

embedding nodes:   0%|          | 0/1248 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

### I had issues with the RPM here. I tried reducing the test size but it didn´t work. I finally used 'raise_excepctions=False' but it seemed to complete the work

In [27]:
testset.test_data[0]

DataRow(question='What factors contribute to inconsistent spikes and drops in demand for our products?', contexts=['impact demand for our products, including by leading to inconsistent spikes and drops in demand. For example, several years ago, our Gaming GPUs began to\nbe used for mining digital currencies, such as Ethereum. It is difficult for us to estimate with any reasonable degree of precision the past or current impact of\ncryptocurrency mining, or forecast the future impact of cryptocurrency mining, on demand for our products. Volatility in the cryptocurrency market, including new\ncompute technologies, price changes in cryptocurrencies, government cryptocurrency policies and regulations, new cryptocurrency standards and changes in'], ground_truth='mining digital currencies, such as Ethereum', evolution_type='simple')

# Generate the responses to the test questions

In [28]:
test_df = testset.to_pandas()

In [29]:
test_df

,question,contexts,ground_truth,evolution_type,episode_done
0,What factors contribute to inconsistent spikes...,"[impact demand for our products, including by ...","mining digital currencies, such as Ethereum",simple,True
1,What percentage of total revenue for fiscal ye...,[Table of Contents\nAll Other operating loss -...,56%,simple,True
2,How would export controls on networking produc...,[controls to restrict additional gaming produc...,Export controls on networking products would n...,reasoning,True
3,Which countries are affected by export restric...,[supercomputing industries. These restrictions...,China and Russia are affected by export restri...,reasoning,True
4,"""How do we account for impairment of non-marke...",[based on quantitative and qualitative factors...,We write down the investment to its fair value...,multi_context,True
5,What factors can affect a business's effective...,[adjustments to income taxes upon finalization...,adjustments to income taxes upon finalization ...,multi_context,True
6,What are the consequences of violating export ...,[and generally fulfill our contractual obligat...,If we were ever found to have violated export ...,multi_context,True
7,What legal challenges do companies like ours f...,[expensed using an accelerated amortization mo...,Companies like ours face legal challenges in r...,multi_context,True
8,How does the integration of cloud-based infras...,"[criteria, our financial results may be harmed...",The integration of cloud-based infrastructure ...,multi_context,True
9,Which integrated circuits are affected by expo...,[supercomputing industries. These restrictions...,The A100 and H100 integrated circuits are affe...,reasoning,True


In [30]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [31]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

In [32]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [33]:
response_dataset[0]

{'question': 'What factors contribute to inconsistent spikes and drops in demand for our products?',
 'answer': 'Factors that contribute to inconsistent spikes and drops in demand for products include changes in customer requirements, new product introductions resulting in less demand for existing products, failure to estimate customer demand properly, ordering in advance of historical lead-times, government regulations, and an increase in demand for competitive products.',
 'contexts': ['Many additional factors have caused and/or could in the future cause us to either underestimate or overestimate our customers’ future demand for our products,\nor otherwise cause a mismatch between supply and demand for our products and impact the timing and volume of our revenue, including:\n•\nchanges in product development cycles and time to market;\n•\ncompeting technologies and competitor product releases and announcements;\n•\nchanges in business and economic conditions resulting in decreased en

# Evaluation

In [34]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [35]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [37]:
results

{'faithfulness': 0.9400, 'answer_relevancy': 0.9377, 'context_recall': 1.0000, 'context_precision': 0.8417, 'answer_correctness': 0.7421}

In [36]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What factors contribute to inconsistent spikes...,Factors that contribute to inconsistent spikes...,[Many additional factors have caused and/or co...,"mining digital currencies, such as Ethereum",1.0,0.986114,1.0,0.250000,0.174126
1,What percentage of total revenue for fiscal ye...,56%,"[10 \n(2)\n— \nTotal\n$\n(4,890)\n$\n(5,411)\n...",56%,1.0,1.000000,1.0,0.833333,1.000000
2,How would export controls on networking produc...,Export controls on networking products could d...,[controls to restrict additional gaming produc...,Export controls on networking products would n...,1.0,0.936295,1.0,1.000000,0.792353
3,Which countries are affected by export restric...,China and Russia,[supercomputing industries. These restrictions...,China and Russia are affected by export restri...,1.0,0.915674,1.0,0.833333,0.965775
4,"""How do we account for impairment of non-marke...",We write down the investment to its fair value...,[based on quantitative and qualitative factors...,We write down the investment to its fair value...,1.0,0.857290,1.0,1.000000,0.740468
5,What factors can affect a business's effective...,Factors that can affect a business's effective...,[adjustments to income taxes upon finalization...,adjustments to income taxes upon finalization ...,1.0,0.952172,1.0,1.000000,0.556940
6,What are the consequences of violating export ...,Violating export control laws or sanctions cou...,[and generally fulfill our contractual obligat...,If we were ever found to have violated export ...,1.0,0.946798,1.0,1.000000,0.739889
7,What legal challenges do companies like ours f...,Companies like yours face legal challenges rel...,[and cash repatriation restrictions; data priv...,Companies like ours face legal challenges in r...,0.4,0.929927,1.0,0.500000,0.991025
8,How does the integration of cloud-based infras...,The integration of cloud-based infrastructure ...,[to some of ours and can use or develop their ...,The integration of cloud-based infrastructure ...,1.0,0.968177,1.0,1.000000,0.489370
9,Which integrated circuits are affected by expo...,A100 and H100 integrated circuits,[supercomputing industries. These restrictions...,The A100 and H100 integrated circuits are affe...,1.0,0.884453,1.0,1.000000,0.970987


# Question #1

In [38]:
question = "Who is the E-VP, Operations - and how old are they"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

Debora Shoquist is the Executive Vice President, Operations, and she is 69 years old.
[Document(page_content='supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName\nAge\nPosition\nJen-Hsun Huang\n60\nPresident and Chief Executive Officer\nColette M. Kress\n56\nExecutive Vice President and Chief Financial Officer\nAjay K. Puri\n69\nExecutive Vice President, Worldwide Field Operations\nDebora Shoquist\n69', metadata={'source': 'Nvidia_10k_Filings.pdf', 'file_path': 'Nvidia_10k_Filings.pdf', 'page': 11, 'total_pages': 96, 'format': 'PDF 1.4', 'title': '0001045810-24-000029', 'author': 'EDGA

# Question #2

In [39]:
question = "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

$3,539
[Document(page_content='Table of Contents\nNVIDIA Corporation and Subsidiaries\nNotes to the Consolidated Financial Statements\n(Continued)\nNote 7 - Amortizable Intangible Assets\nThe components of our amortizable intangible assets are as follows:\n \nJan 28, 2024\nJan 29, 2023\n \nGross\nCarrying\nAmount\nAccumulated\nAmortization\nNet \nCarrying\nAmount\nGross\nCarrying\nAmount\nAccumulated\nAmortization\nNet \nCarrying\nAmount\n \n(In millions)\nAcquisition-related intangible\nassets (1)\n$\n2,642 \n$\n(1,720)\n$\n922 \n$\n3,093 \n$\n(1,614)\n$\n1,479 \nPatents and licensed technology\n449 \n(259)\n190 \n446 \n(249)\n197 \nTotal intangible assets\n$\n3,091 \n$\n(1,979)\n$\n1,112 \n$\n3,539 \n$\n(1,863)\n$\n1,676', metadata={'source': 'Nvidia_10k_Filings.pdf', 'file_path': 'Nvidia_10k_Filings.pdf', 'page': 63, 'total_pages': 96, 'format': 'PDF 1.4', 'title': '0001045810-24-000029', 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'subject': 'Form 10-K filed o